<a href="https://colab.research.google.com/github/itsSuryanshu/3-pre-trained-CNNmodels-LungDisease/blob/main/CNNmodelsPredictInterface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
!pip install gradio

In [2]:
import os
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

In [62]:
# Download the models
!gdown 1-1yxqc1wIZ5BCkMSMHX3ckDutx7W53vh
!gdown 1-2UXYM3nRDFMuDdz16Ch--HysRn34dre
!gdown 1-4fuab4lPUPuzwo8frTf8dMbPS1-RagB
!gdown 1-1PiPw1_Ki3dROe4STsqBlUtNccwBHRg

In [4]:
# Load up the models
cnn_models = {}
for file in os.listdir("/content"):
  if file.endswith(".keras"):
    model_name = os.path.basename(file).split(".")[0]
    model_path = os.path.join(os.getcwd(), file)
    cnn_models[model_name] = load_model(model_path)

In [5]:
# Check the loaded models
cnn_models

{'densenet121_model': <Functional name=functional_7, built=True>,
 'KerasModel': <Functional name=functional, built=True>,
 'vgg16_model': <Functional name=functional_1, built=True>,
 'resnet50v2_model': <Functional name=functional_3, built=True>}

In [57]:
# Image prediction function
def img_prediction(img, model_name):
  try:
    # Pre-process the image
    if model_name == "KerasModel":
      resize_img = tf.image.resize(img, (180, 180))
      img_array = tf.keras.utils.img_to_array(resize_img)
      # final = np.expand_dims(resize_img/255, 0)
      final = tf.expand_dims(img_array, 0)
    else:
      resize_img = tf.image.resize(img, (224, 224))
      final = np.expand_dims(resize_img/255, 0)

    # Input image into model for prediction
    model = cnn_models[model_name]
    prediction = model.predict(final)

    # Process and return the result
    if model_name == "KerasModel":
      result = float(tf.sigmoid(prediction[0][0]))
      if result > 0.5:
        return f"The X-ray results show it is {round(result * 100, 2)}% Viral Pneumonia"
      else:
        return f"The X-ray results show it is {100 - round(result * 100, 2)}% Normal"
    else:
      result = np.array(prediction[0])
      if result[0] > result[1]:
        return f"The X-ray results show it is {round(result[0]*100, 2)}% Normal"
      else:
        return f"The X-ray results show it is {round(result[1]*100, 2)}% Viral Pneumonia"
  except Exception as e:
    #!Error!: Looks like there's a problem with your inputs! Check that your image is of type 'jpeg', 'jpg' or 'png' and that you have selected a model.
    return f"Error: {e}"

In [58]:
demo = gr.Interface(
    fn = img_prediction,
    inputs = [
        gr.Image(type="numpy"),
        gr.Dropdown(label="Select Model", choices=list(cnn_models.keys()))
    ],
    outputs = gr.Textbox(label="Prediction:"),
    title = "Keras CNN Lung Disease Classification"
)

In [61]:
demo.launch()